In [2]:
import ipywidgets as widgets
_out = widgets.Output()
with _out:
    from ipynb.fs.full.invoiceDemoLibrary import *

In [3]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]
_keys = calc_keys(assets)

# Demo Functions

In [4]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            #result.append(get_cert_cn(person['name']))
            result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

In [5]:
def get_current_user_with_role(rolename):    
    if rolename == 'Buyer':
        return buyer
    elif  rolename == 'Seller':   
        return owner
    elif  rolename == 'Factor':   
        return factor
    elif  rolename == 'TaxInspector':    
        return taxinspector
    
def get_current_peer_with_role(rolename):    
    if rolename == 'Buyer':
        return peer_buyer
    elif  rolename == 'Seller':   
        return peer_owner
    elif  rolename == 'Factor':   
        return peer_factor
    elif  rolename == 'TaxInspector':    
        return peer_taxinspector    

def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    disable_all_fields()
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [6]:
def create_dropdown_owner(_widget):
    _certs=[_widget.container.fields['owner'].widget.value]
    _certs = _certs + get_person_certs_with_role('Factor',roles)
    return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

def replace_owner_field_by_dropdown(_widget):
    _owners = create_dropdown_owner(_widget)
    _widget.container.fields['owner'] = Field(widget=_owners,path=_widget.container.fields['owner'].path)
    _widget.redisplay()

### role buyer

In [7]:
def init_fields_buyer(user=None,peer=None):
    #peer=get_peer_of_user(user)

    def received_changed(b):
        if b['name'] == 'value':
            receivedInvoice(peer, id=id, user=user)[1]

    def receivedOrder_changed(b):
        if b['name'] == 'value':
            receivedOrder(peer, id=id, user=user)[1]    

    _widget.container.fields['received'].widget.observe(received_changed)
    _widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)
    enable_fields_for_role('Buyer')

### role owner / seller

In [8]:
def init_fields_seller(user=None,peer=None):
    #peer=get_peer_of_user(user)
    
    def sold_changed(b):
        if b['name'] == 'value':
            name=_widget.container.fields['owner'].widget.value
            transferInvoice(peer, id=id,name=name, user=user)[1]

    def claimPaid_changed(b):
        if b['name'] == 'value':
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Seller')    
    _widget.container.fields['sold'].widget.observe(sold_changed)
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   
    replace_owner_field_by_dropdown(_widget)

### role factor (owner)

In [9]:
def init_fields_factor(user=None,peer=None):
    #peer=get_peer_of_user(user)
    
    def claimPaid_changed(b):
        if b['name'] == 'value':  
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Factor')    
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   

### role tax inspector

In [10]:
def init_fields_taxinspector(user=None,peer=None):
    #peer=get_peer_of_user(user)

    def taxReceived_changed(b):
        if b['name'] == 'value':
            taxReceived(peer, id=id, user=user)

    _widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)
    enable_fields_for_role('TaxInspector')

In [21]:
#invoice widget

#def asset_selected(x): 
#    global asset
#    asset = listInvoice(shop_owner_peer, id=assetOptions.value, user=shop_owner)[0]
    
#assetOptions = widgets.Dropdown(options=_keys)
#assetOptions.observe(asset_selected, names='value')

def widget_load_asset(widget,asset):
    widget.output.clear_output()
    widget.container = FieldContainer()
    with widget.output:
        widget.addChilds(asset,'/', hideUnused=True)

def reload_clicked(i):
    global buyer,peer_buyer
    global asset
    
    
    user = get_current_user_with_role(roleOptions.value)
    peer = get_current_peer_with_role(roleOptions.value)
    #print(user)
    #asset = listInvoice(shop_owner_peer, id, user=shop_owner)[0]
    asset = listInvoice(peer, id, user=user)[0]
    #print(asset['buyer'])
    with _out:
        buyer=get_cert_cn(asset['buyer'])
        peer_buyer=get_peer_of_user(buyer)
        
    widget_load_asset(_widget,asset)
    role_selected('')  

def role_selected(x): 
    if roleOptions.value == 'Buyer':
        init_fields_buyer(user=buyer,peer=peer_buyer)
    elif roleOptions.value == 'Seller':
        init_fields_seller(user=owner,peer=peer_owner)
    elif roleOptions.value == 'TaxInspector':    
        init_fields_taxinspector(user=taxinspector,peer=peer_taxinspector)


def asset_selected(x): 
    global id
    id=assetOptions.value
    #asset = listInvoice(shop_owner_peer, id=assetOptions.value, user=shop_owner)[0]
    #roleOptions.value == 'Seller'
    reload_clicked('')
    
    
def create_invoice_widget():    
    #global _widget


    _widget = facturx_widgets(asset,'/', hideUnused=True, hide=True) 
    
    #display(_widget)

    HBox = widgets.HBox([_widget.output, options,assetOptions]) 
    display(HBox)

    assetOptions.value='asset14'
    
    return _widget

roleOptions = widgets.Dropdown(options=usedRoles)
#roleOptions.observe(role_selected, names='value')
roleOptions.observe(reload_clicked, names='value')
        
btn = widgets.Button(description=f'From Blockchain', disabled=False,button_style='success', icon='check')    
btn.on_click(reload_clicked)
#display (btn)

assetOptions = widgets.Dropdown(options=_keys)
assetOptions.observe(asset_selected, names='value')
options = widgets.VBox([roleOptions,btn])




In [12]:
_invoice = invoices.iloc[14]

# Rechnung

In [13]:
facturx_widgets(_invoice,'/supplyChainTradeTransaction', hideUnused=True, hide=True).output

Output()

# Adressen

In [14]:
create_address_widget(_invoice)

# Rollen

In [15]:
create_role_widget(shop_owner_roles)

Output()

# Invoice Widget

In [22]:
_widget = create_invoice_widget()
asset_selected('')